# Recurrent neural networks with Keras


In this notebook we will build RNN models. These models have been initially designed for sequence-to-sequence modeling such as text, hence we will see that using them for time-series modeling is not yet straightforward.

In [3]:
import numpy as np
from tensorflow.keras.layers import Input, Dense, SimpleRNN
from tensorflow.keras.models import Sequential, Model

/home/alexander/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/alexander/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/alexander/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/alexander/.local/lib/python3.6/site-packages/tensorflow/python/framework/dty

## Warm-up 

Let's first recall how we would build a standard ANN model. There are two ways. Assume the input shape is 5.

In [7]:
## first way : with Sequential for simple models

model = Sequential([
    Dense(10, input_shape=[5,]),
    Dense(1, activation="sigmoid")
])

## display model sunmmary
model.summary()

## prepare some dummy data. NB the reshape!
data = np.array([0.1, 0.2, 0.3, 0.4, 0.5]).reshape((1,5)) ## (observations, features)

## make a prediction
model.predict(data)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 10)                60        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 11        
Total params: 71
Trainable params: 71
Non-trainable params: 0
_________________________________________________________________


array([[0.39797702]], dtype=float32)

In [11]:
## second way : with Model for more complex models
input1 = Input(shape=(5))
h1 = Dense(units=10)(input1)
outputs1 = Dense(1, activation="sigmoid")(h1)


model = Model(input1, outputs1)
model.summary()
## TODO
model.predict(data)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 5)]               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                60        
_________________________________________________________________
dense_14 (Dense)             (None, 1)                 11        
Total params: 71
Trainable params: 71
Non-trainable params: 0
_________________________________________________________________


array([[0.37310147]], dtype=float32)

## RNN training

The tricky part with RNN is that the input vector is a fixed 2-dimensional where:
* the first coordinate is the number of ordered elements per sequence (= observations such as days)
* the second element is the number of features per element (= number of stocks)

Therefore, we need to set the number of days on which the model will be trained. At the same time, the training data must 3-dimensional where
* the first coordinate the number of sequences (= time series)
* the second and third coordinates are as above

because one model can be trained on many sequences (=times series)

In [2]:
## assume that we have a time series with 5 dates
input1 = Input(shape=(None, 5))

h1 = SimpleRNN(10,activation="relu", return_sequences=True)(input1)
output1 = Dense(1, activation="sigmoid")(h1)

model = Model(input1, output1)
x = np.random.normal(0,1,(1,10,5))
## prepare some dummy data. NB the reshape!
data = np.array([0.1, 0.2, 0.3, 0.4, 0.5]).reshape((1,5,1))## (observations, timestamps, features)

## make a prediction
model.summary()
model.predict(x)

NameError: name 'Input' is not defined

In [5]:
input1 = Input(shape=(None, 5))
input2 = Input(shape=(10,))

h1, st = SimpleRNN(10,activation="relu", return_sequences=True, return_state=True)(input1, initial_state=input2)
output1 = Dense(1, activation="sigmoid")(h1)

model = Model([input1, input2], [output1, st])
x = [np.random.normal(0,1,(1,10,5)),
     np.random.normal(0,1,(1,10))]
## prepare some dummy data. NB the reshape!
data = np.array([0.1, 0.2, 0.3, 0.4, 0.5]).reshape((1,5,1))## (observations, timestamps, features)

## make a prediction
model.summary()
model.predict(x)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 5)]    0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 10)]         0                                            
__________________________________________________________________________________________________
simple_rnn_1 (SimpleRNN)        [(None, None, 10), ( 160         input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, None, 1)      11          simple_rnn_1[0][0]           

[array([[[0.80337036],
         [0.4059868 ],
         [0.9819258 ],
         [0.8973104 ],
         [0.9919016 ],
         [0.94812787],
         [0.9848033 ],
         [0.9961839 ],
         [0.9946755 ],
         [0.9999932 ]]], dtype=float32),
 array([[1.1046875 , 4.3021874 , 0.51174426, 0.42728996, 0.10060626,
         5.0671344 , 3.4954302 , 6.920345  , 2.14085   , 2.9147596 ]],
       dtype=float32)]

In applications, we might be interested to also ouput the hidden state values, so let's see how we can do that.

In [ ]:
## TODO

## RNN testing and live

At testing time, we may want to:
* not run model on the entire dataset for each prediction!
* use the last hidden state

Indeed, each time a prediction is made as above, Keras use the default initial state NOT the latest.

In [28]:
input1 = Input(shape=(1, 1)) ## x_t : pass just one observation
input2 = Input(shape=(1)) ## h_t : assuming one hidden state

## build the model
model_onestep = None

## transfer the weights from the trained model
model_onestep.set_weights([w for w in model.get_weights()])

## prepare data and make a prediction
data = [np.array(0.6).reshape((1,1,1)), ## x_t
        np.array(1.0).reshape((1,1))    ## h_t
       ]

AttributeError: 'NoneType' object has no attribute 'set_weights'

## Custom RNN layer (advanced topic)

Below is a sample code illustrating how you can build your own RNN layer.

In [29]:
from tensorflow.keras.layers import RNN
from tensorflow import keras
K = keras.backend

class MySimpleRNNCell(keras.layers.Layer):
    
    ## cell initialization
    def __init__(self, units, **kwargs):
        self.units = units
        self.state_size = units
        super(MySimpleRNNCell, self).__init__(**kwargs)

    ## prepare the variables given a specific input tensor
    def build(self, input_shape):
        self.kernel = self.add_weight(
            shape=(input_shape[-1], self.units),
            name='kernel')
        self.recurrent_kernel = self.add_weight(
            shape=(self.units, self.units),
            name='recurrent_kernel')
        self.bias = self.add_weight(
            shape=(1,self.units),
            name='bias')
        self.built = True
    
    ## this is smart function transforming input and states
    def call(self, inputs, states):
        prev_output = states[0]
        h = K.dot(inputs, self.kernel) + self.bias
        h = h + K.dot(prev_output, self.recurrent_kernel)
        output = keras.activations.relu(h)
        return output, [output]

## initialize the custom layer
my_cell = MySimpleRNNCell(units=1) ## call the function "__init__"
my_layer = RNN(my_cell) ## RNN manage the states

## build the model
input1 = keras.Input((5,1))
output = my_layer(input1) ## call the function "build"

## make a prediction
data = np.array([0.1, 0.2, 0.3, 0.4, 0.5]).reshape((1,5,1))
